In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

#Branch Schema
branch_schema = StructType([
    StructField("BRANCH_CODE", IntegerType(), True),
    StructField("BRANCH_NAME", StringType(), True),
    StructField("BRANCH_STREET", StringType(), True),
    StructField("BRANCH_CITY", StringType(), True),
    StructField("BRANCH_STATE", StringType(), True),
    StructField("BRANCH_ZIP", IntegerType(), True),
    StructField("BRANCH_PHONE", StringType(), True),
    StructField("LAST_UPDATED", TimestampType(), True)
])

spark = SparkSession.builder.appName("JsonExtraction").getOrCreate()

branch_data = spark.read.json("cdw_sapp_branch.json")





In [2]:
branch_data.createOrReplaceTempView("branch_data")

In [3]:
#Extracting the transformed Branch Data
transformed_branch_data = spark.sql("""
    SELECT
        CAST(BRANCH_CODE AS INT) AS BRANCH_CODE,
        BRANCH_NAME AS BRANCH_NAME,
        BRANCH_STREET AS BRANCH_STREET,
        BRANCH_CITY AS BRANCH_CITY,
        BRANCH_STATE AS BRANCH_STATE,
        CASE
            WHEN BRANCH_ZIP IS NULL THEN 99999
            ELSE CAST(BRANCH_ZIP AS INT)
        END AS BRANCH_ZIP,
        CASE
            WHEN BRANCH_PHONE RLIKE '\\d{10}'
            THEN CONCAT('(', SUBSTR(BRANCH_PHONE, 1, 3), ')', SUBSTR(BRANCH_PHONE, 4, 3), '-', SUBSTR(BRANCH_PHONE, 7, 4))
            ELSE BRANCH_PHONE
        END AS BRANCH_PHONE,
        LAST_UPDATED AS LAST_UPDATED
    FROM branch_data
""")

transformed_branch_data.show()

+-----------+------------+-------------------+-----------------+------------+----------+------------+--------------------+
|BRANCH_CODE| BRANCH_NAME|      BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP|BRANCH_PHONE|        LAST_UPDATED|
+-----------+------------+-------------------+-----------------+------------+----------+------------+--------------------+
|          1|Example Bank|       Bridle Court|        Lakeville|          MN|     55044|  1234565276|2018-04-18T16:51:...|
|          2|Example Bank|  Washington Street|          Huntley|          IL|     60142|  1234618993|2018-04-18T16:51:...|
|          3|Example Bank|      Warren Street|SouthRichmondHill|          NY|     11419|  1234985926|2018-04-18T16:51:...|
|          4|Example Bank|   Cleveland Street|       Middleburg|          FL|     32068|  1234663064|2018-04-18T16:51:...|
|          5|Example Bank|        14th Street|    KingOfPrussia|          PA|     19406|  1234849701|2018-04-18T16:51:...|
|          7|Exa

In [4]:
#Customer Schema

schema_customer = StructType([
    StructField("SSN", IntegerType(), True),
    StructField("FIRST_NAME", StringType(), True),
    StructField("MIDDLE_NAME", StringType(), True),
    StructField("LAST_NAME", StringType(), True),
    StructField("CREDIT_CARD_NO", StringType(), True),
    StructField("STREET_NAME", StringType(), True),
    StructField("APT_NO", StringType(), True),
    StructField("CUST_CITY", StringType(), True),
    StructField("CUST_STATE", StringType(), True),
    StructField("CUST_COUNTRY", StringType(), True),
    StructField("CUST_ZIP", IntegerType(), True),
    StructField("CUST_PHONE", StringType(), True),
    StructField("CUST_EMAIL", StringType(), True),
    StructField("LAST_UPDATED", TimestampType(), True)
])


In [5]:
#Reading json file with schema
customer_data = spark.read.schema(schema_customer).json("cdw_sapp_custmer.json") 

In [6]:
#Temp view for customer data
customer_data.createOrReplaceTempView("customer_data")

In [7]:
# Write SQL queries based on the qualified column names
transformed_customer_data = spark.sql("""
    SELECT
        CAST(customer_data.SSN AS INT) AS SSN,
        INITCAP(customer_data.FIRST_NAME) AS FIRST_NAME,
        LOWER(customer_data.MIDDLE_NAME) AS MIDDLE_NAME,
        INITCAP(customer_data.LAST_NAME) AS LAST_NAME,
        customer_data.CREDIT_CARD_NO AS Credit_card_no,
        CONCAT_WS(', ', customer_data.APT_NO, customer_data.STREET_NAME) AS FULL_STREET_ADDRESS,
        customer_data.CUST_CITY AS CUST_CITY,
        customer_data.CUST_STATE AS CUST_STATE,
        customer_data.CUST_COUNTRY AS CUST_COUNTRY,
        CAST(COALESCE(customer_data.CUST_ZIP, 99999) AS INT) AS CUST_ZIP,
        CASE
            WHEN customer_data.CUST_PHONE RLIKE '\\d{10}'
            THEN CONCAT('(', SUBSTR(customer_data.CUST_PHONE, 1, 3), ')', SUBSTR(customer_data.CUST_PHONE, 4, 3), '-', SUBSTR(customer_data.CUST_PHONE, 7, 4))
            ELSE customer_data.CUST_PHONE
        END AS CUST_PHONE,
        customer_data.CUST_EMAIL AS CUST_EMAIL,
        customer_data.LAST_UPDATED AS LAST_UPDATED
    FROM customer_data
""")


transformed_customer_data.show()

+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+----------+--------------------+-------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  Credit_card_no| FULL_STREET_ADDRESS|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|CUST_PHONE|          CUST_EMAIL|       LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+----------+--------------------+-------------------+
|123456100|      Alec|         wm|   Hooper|4210653310061055|656, Main Street ...|     Natchez|        MS|United States|   99999|   1237818| AHooper@example.com|2018-04-21 09:49:02|
|123453023|      Etta|    brendan|   Holman|4210653310102868|  829, Redwood Drive|Wethersfield|        CT|United States|   99999|   1238933| EHolman@example.com|2018-04-21 09:49:02|
|123454487|    Wilber|   ezequiel|   Dunham|4210653310116272|683, 12th Street ...|     Hun

In [8]:
#Schema for Credit card
schema_credit_card = StructType([
    StructField("CUST_CC_NO", StringType(), True),
    StructField("TIMEID", StringType(), True),
    StructField("CUST_SSN", IntegerType(), True),
    StructField("BRANCH_CODE", IntegerType(), True),
    StructField("TRANSACTION_TYPE", StringType(), True),
    StructField("TRANSACTION_VALUE", DoubleType(), True),
    StructField("TRANSACTION_ID", IntegerType(), True)
])

In [9]:
# Read the JSON file with the defined schema
credit_card_data = spark.read.schema(schema_credit_card).json("cdw_sapp_credit.json")

In [10]:
# Create a temporary view for credit card data
credit_card_data.createOrReplaceTempView("credit_card_data")

In [11]:
# Write SQL queries based on the qualified column names
transformed_credit_card_data = spark.sql("""
    SELECT
        CUST_CC_NO AS CUST_CC_NO,
        TIMEID AS TIMEID,
        CAST(CUST_SSN AS INT) AS CUST_SSN,
        CAST(BRANCH_CODE AS INT) AS BRANCH_CODE,
        TRANSACTION_TYPE AS TRANSACTION_TYPE,
        CAST(TRANSACTION_VALUE AS DOUBLE) AS TRANSACTION_VALUE,
        CAST(TRANSACTION_ID AS INT) AS TRANSACTION_ID
    FROM credit_card_data
""")

transformed_credit_card_data.show()

+----------+------+---------+-----------+----------------+-----------------+--------------+
|CUST_CC_NO|TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------+------+---------+-----------+----------------+-----------------+--------------+
|      null|  null|123459988|        114|       Education|             78.9|             1|
|      null|  null|123459988|         35|   Entertainment|            14.24|             2|
|      null|  null|123459988|        160|         Grocery|             56.7|             3|
|      null|  null|123459988|        114|   Entertainment|            59.73|             4|
|      null|  null|123459988|         93|             Gas|             3.59|             5|
|      null|  null|123459988|        164|       Education|             6.89|             6|
|      null|  null|123459988|        119|   Entertainment|            43.39|             7|
|      null|  null|123459988|         23|             Gas|            95.39|    

In [12]:
import mysql.connector
import secrets


spark = SparkSession.builder.appName("CreditCardSystemLoader").getOrCreate()

# Load the transformed data into DataFrames (replace with your DataFrames)
branch_data = transformed_branch_data
credit_card_data = transformed_credit_card_data
customer_data = transformed_customer_data


db = mysql.connector.connect(
    host = "localhost",
    user = secrets.mysql_username,
    passwd = secrets.mysql_password,
    database = "creditcard_capstone" #Database in mysql

)


# Write DataFrames to MySQL tables using the existing MySQL connection
branch_data.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_BRANCH") \
    .option("mode", "overwrite") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("user", secrets.mysql_username) \
    .option("password", secrets.mysql_password) \
    .save()
credit_card_data.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
    .option("mode", "overwrite") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("user", secrets.mysql_username) \
    .option("password", secrets.mysql_password) \
    .save()
customer_data.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CUSTOMER") \
    .option("mode", "overwrite") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("user", secrets.mysql_username) \
    .option("password", secrets.mysql_password) \
    .save()
# Stop the Spark session
spark.stop()
# Close the MySQL connection
db.close()

AnalysisException: Table or view 'CDW_SAPP_BRANCH' already exists. SaveMode: ErrorIfExists.

In [13]:
#Req 2.1
import mysql.connector
# Define your database connection details
db = mysql.connector.connect(
    host = "localhost",
    user = secrets.mysql_username,
    passwd = secrets.mysql_password,
    database = "creditcard_capstone" #Database in mysql

)
cursor = db.cursor()
# Function to display transactions by zip code, month, and year
def display_transactions_by_zip_month_year(zip_code, month, year):
    # SQL query to retrieve transactions by zip code, month, and year
    query = """
    SELECT * FROM CDW_SAPP_CREDIT_CARD
    WHERE YEAR(TIMEID) = %s AND MONTH(TIMEID) = %s AND CUST_ZIP = %s
    ORDER BY TIMEID DESC;
    """
    cursor.execute(query, (year, month, zip_code))
    transactions = cursor.fetchall()
    # Display the results
    print("Transactions by Zip Code, Month, and Year:")
    for transaction in transactions:
        print(transaction)
# Function to display number and total values of transactions for a given type
def display_transactions_by_type(transaction_type):
    # SQL query to retrieve transactions by type
    query = """
    SELECT COUNT(*) AS transaction_count, SUM(TRANSACTION_VALUE) AS total_value
    FROM CDW_SAPP_CREDIT_CARD
    WHERE TRANSACTION_TYPE = %s;
    """
    cursor.execute(query, (transaction_type,))
    result = cursor.fetchone()
    # Display the results
    print("Number of Transactions by Type:", result[0])
    print("Total Value of Transactions by Type:", result[1])
# Function to display total number and total values of transactions for branches in a given state
def display_transactions_by_state(branch_state):
    # SQL query to retrieve transactions by state
    query = """
    SELECT BRANCH_CODE, COUNT(*) AS transaction_count, SUM(TRANSACTION_VALUE) AS total_value
    FROM CDW_SAPP_CREDIT_CARD
    WHERE BRANCH_CODE IN (
        SELECT BRANCH_CODE FROM CDW_SAPP_BRANCH WHERE BRANCH_STATE = %s
    )
    GROUP BY BRANCH_CODE;
    """
    cursor.execute(query, (branch_state,))
    results = cursor.fetchall()
    # Display the results
    print("Transactions by State:", branch_state)
    for result in results:
        print("Branch Code:", result[0])
        print("Number of Transactions:", result[1])
        print("Total Value of Transactions:", result[2])
# Main program
if __name__ == "__main":
    while True:
        print("Transaction Details Module:")
        print("1. Display Transactions by Zip Code, Month, and Year")
        print("2. Display Number and Total Values of Transactions by Type")
        print("3. Display Total Number and Total Values of Transactions by Branch State")
        print("4. Quit")
        choice = input("Enter your choice: ")
        if choice == "1":
            zip_code = input("Enter Zip Code: ")
            month = input("Enter Month (numeric): ")
            year = input("Enter Year (YYYY): ")
            display_transactions_by_zip_month_year(zip_code, month, year)
        elif choice == "2":
            transaction_type = input("Enter Transaction Type: ")
            display_transactions_by_type(transaction_type)
        elif choice == "3":
            branch_state = input("Enter Branch State: ")
            display_transactions_by_state(branch_state)
        elif choice == "4":
            break
        else:
            print("Invalid choice. Please enter a valid option.")
    # Close the database connection
    cursor.close()
    db.close()

In [14]:
#Req 2.2
import mysql.connector
# Define your database connection details and connect
db = mysql.connector.connect(
    host = "localhost",
    user = secrets.mysql_username,
    passwd = secrets.mysql_password,
    database = "creditcard_capstone" #Database in mysql
)
cursor = db.cursor()
# Function to check existing account details of a customer
def check_customer_details(customer_id):
    # SQL query to retrieve customer details
    query = "SELECT * FROM CDW_SAPP_CUSTOMER WHERE SSN = %s"
    cursor.execute(query, (customer_id,))
    customer_details = cursor.fetchone()
    # Display customer details
    if customer_details:
        print("Customer Details:")
        print("SSN:", customer_details[0])
        print("First Name:", customer_details[1])
        print("Last Name:", customer_details[2])
        print("Street Address:", customer_details[3])
        print("City:", customer_details[4])
        print("State:", customer_details[5])
        print("Country:", customer_details[6])
        print("ZIP:", customer_details[7])
        print("Phone:", customer_details[8])
        print("Email:", customer_details[9])
    else:
        print("Customer not found.")
# Function to modify existing account details of a customer
def modify_customer_details(customer_id, new_email):
    # SQL query to update customer email
    query = "UPDATE CDW_SAPP_CUSTOMER SET CUST_EMAIL = %s WHERE SSN = %s"
    cursor.execute(query, (new_email, customer_id))
    db.commit()
    print("Customer email updated successfully.")
# Function to generate a monthly bill for a credit card
def generate_monthly_bill(credit_card_number, month, year):
    # SQL query to retrieve monthly bill details
    query = """
    SELECT * FROM CDW_SAPP_CREDIT_CARD
    WHERE YEAR(TIMEID) = %s AND MONTH(TIMEID) = %s AND CUST_CC_NO = %s
    ORDER BY TIMEID DESC;
    """
    cursor.execute(query, (year, month, credit_card_number))
    monthly_bill = cursor.fetchall()
    # Display the monthly bill
    if monthly_bill:
        print("Monthly Bill for Credit Card Number:", credit_card_number)
        for transaction in monthly_bill:
            print(transaction)
    else:
        print("No transactions found for the specified month and year.")
# Function to display customer transactions between two dates
def display_transactions_between_dates(customer_id, start_date, end_date):
    # SQL query to retrieve transactions between two dates
    query = """
    SELECT * FROM CDW_SAPP_CREDIT_CARD
    WHERE TIMEID BETWEEN %s AND %s AND CUST_CC_NO = %s
    ORDER BY TIMEID DESC;
    """
    cursor.execute(query, (start_date, end_date, customer_id))
    transactions = cursor.fetchall()
    # Display transactions
    if transactions:
        print("Transactions between", start_date, "and", end_date)
        for transaction in transactions:
            print(transaction)
    else:
        print("No transactions found between the specified dates.")
# Main program
if __name__ == "__main":
    while True:
        print("Customer Details Module:")
        print("1. Check Customer Details")
        print("2. Modify Customer Email")
        print("3. Generate Monthly Bill")
        print("4. Display Customer Transactions Between Two Dates")
        print("5. Quit")
        choice = input("Enter your choice: ")
        if choice == "1":
            customer_id = input("Enter Customer SSN: ")
            check_customer_details(customer_id)
        elif choice == "2":
            customer_id = input("Enter Customer SSN: ")
            new_email = input("Enter New Email: ")
            modify_customer_details(customer_id, new_email)
        elif choice == "3":
            credit_card_number = input("Enter Credit Card Number: ")
            month = input("Enter Month (numeric): ")
            year = input("Enter Year (YYYY): ")
            generate_monthly_bill(credit_card_number, month, year)
        elif choice == "4":
            customer_id = input("Enter Customer SSN: ")
            start_date = input("Enter Start Date (YYYYMMDD): ")
            end_date = input("Enter End Date (YYYYMMDD): ")
            display_transactions_between_dates(customer_id, start_date, end_date)
        elif choice == "5":
            break
        else:
            print("Invalid choice. Please enter a valid option.")
    # Close the database connection
    cursor.close()
    db.close()

In [15]:
#Req 3.1
import mysql.connector
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

# Define your database connection details
db = mysql.connector.connect(
    host = "localhost",
    user = secrets.mysql_username,
    passwd = secrets.mysql_password,
    database = "creditcard_capstone" #Database in mysql
)

# SQL query to find transaction type with the highest transaction count
query = """
SELECT TRANSACTION_TYPE, COUNT(*) AS transaction_count
FROM CDW_SAPP_CREDIT_CARD
GROUP BY TRANSACTION_TYPE
ORDER BY transaction_count DESC
LIMIT 1;
"""
# Execute the query and retrieve data
cursor = db.cursor()
cursor.execute(query)
result = cursor.fetchone()
# Extract transaction type and count
transaction_type = result[0]
transaction_count = result[1]
# Plot the result
plt.figure(figsize=(8, 6))
plt.bar(transaction_type, transaction_count)
plt.xlabel("Transaction Type")
plt.ylabel("Transaction Count")
plt.title("Transaction Type with the Highest Transaction Count")
plt.tight_layout()
# Save the visualization to a folder in your GitHub repository with a proper name
plt.savefig("path_to_github_repo/transaction_type_highest_count.png")
plt.show()
# Close the database connection
cursor.close()
db.close()

ImportError: cannot import name randbits

In [16]:
#Req 3.2
import mysql.connector
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd


# SQL query to find the state with a high number of customers
query = """
SELECT CUST_STATE, COUNT(*) AS customer_count
FROM CDW_SAPP_CUSTOMER
GROUP BY CUST_STATE
ORDER BY customer_count DESC
LIMIT 1;
"""
# Execute the query and retrieve data
cursor = db.cursor()
cursor.execute(query)
result = cursor.fetchone()
# Extract state and customer count
state = result[0]
customer_count = result[1]
# Plot the result
plt.figure(figsize=(8, 6))
plt.bar(state, customer_count)
plt.xlabel("State")
plt.ylabel("Customer Count")
plt.title("State with a High Number of Customers")
plt.tight_layout()
# Save the visualization to a folder in your GitHub repository with a proper name
plt.savefig("path_to_github_repo/state_high_customer_count.png")
plt.show()
# Close the database connection
cursor.close()
db.close()

ImportError: cannot import name randbits

In [ ]:
#Req 3.3

import mysql.connector
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd


# SQL query to find the top 10 customers with the highest transaction amounts
query = """
SELECT CUST_SSN, SUM(TRANSACTION_VALUE) AS total_transaction_amount
FROM CDW_SAPP_CREDIT_CARD
GROUP BY CUST_SSN
ORDER BY total_transaction_amount DESC
LIMIT 10;
"""
# Execute the query and retrieve data
cursor = db.cursor()
cursor.execute(query)
result = cursor.fetchall()
# Extract data into separate lists
customer_ssns = [row[0] for row in result]
transaction_amounts = [row[1] for row in result]
# Plot the result
plt.figure(figsize=(10, 6))
plt.bar(customer_ssns, transaction_amounts)
plt.xlabel("Customer SSN")
plt.ylabel("Total Transaction Amount")
plt.title("Top 10 Customers with Highest Transaction Amount")
plt.xticks(rotation=45)
plt.tight_layout()
# Save the visualization to a folder in your GitHub repository with a proper name
plt.savefig("path_to_github_repo/top_10_customers_transaction_amount.png")
plt.show()
# Close the database connection
cursor.close()
db.close()

ImportError: cannot import name randbits

In [ ]:
#Req 4.1, 4.2, 4.3
import requests
import mysql.connector
# API endpoint for the loan application dataset
api_url = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"
# Database connection configuration
db = mysql.connector.connect(
    host = "localhost",
    user = secrets.mysql_username,
    passwd = secrets.mysql_password,
    database = "creditcard_capstone" #Database in mysql
)
# Send a GET request to the API endpoint
response = requests.get(api_url)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    loan_data = response.json()
    # Create a cursor to execute SQL commands
    cursor = db.cursor()
    # Define the table schema for CDW_SAPP_loan_application
    create_table_query = """
    CREATE TABLE IF NOT EXISTS CDW_SAPP_loan_application (
        Application_ID VARCHAR(20) PRIMARY KEY,
        Gender VARCHAR(10),
        Married VARCHAR(10),
        Dependents VARCHAR(10),
        Education VARCHAR(20),
        Self_Employed VARCHAR(10),
        Credit_History INT,
        Property_Area VARCHAR(20),
        Income VARCHAR(20),
        Application_Status VARCHAR(10)
    );
    """
    # Create the CDW-SAPP_loan_application table if it doesn't exist
    cursor.execute(create_table_query)
    # Insert data into the table
    insert_query = """
    INSERT INTO CDW_SAPP_loan_application (
        Application_ID, Gender, Married, Dependents, Education, Self_Employed,
        Credit_History, Property_Area, Income, Application_Status
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    # Iterate through the loan data and insert it into the database
    for record in loan_data:
        data_tuple = (
            record["Application_ID"],
            record["Gender"],
            record["Married"],
            record["Dependents"],
            record["Education"],
            record["Self_Employed"],
            record["Credit_History"],
            record["Property_Area"],
            record["Income"],
            record["Application_Status"],
        )
        cursor.execute(insert_query, data_tuple)
    # Commit the changes and close the cursor and database connection
    db.commit()
    cursor.close()
    db.close()
    print("Loan data loaded into the RDBMS.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

IntegrityError: 1062 (23000): Duplicate entry 'LP001002' for key 'cdw_sapp_loan_application.PRIMARY'

In [ ]:
#Req 5.1

import pandas as pd
import matplotlib.pyplot as plt
# Load data from your MySQL database into a Pandas DataFrame
db = mysql.connector.connect(
    host = "localhost",
    user = secrets.mysql_username,
    passwd = secrets.mysql_password,
    database = "creditcard_capstone" #Database in mysql
)
query = "SELECT * FROM CDW_SAPP_loan_application"
loan_data = pd.read_sql(query, db)
# Filter self-employed applicants
self_employed_applicants = loan_data[loan_data['Self_Employed'] == 'Yes']
# Calculate the percentage of approved applications among self-employed applicants
approved_percentage = (self_employed_applicants['Application_Status'] == 'Y').mean() * 100
# Create a bar chart
plt.bar(['Self-Employed'], [approved_percentage])
plt.ylabel('Percentage Approved')
plt.title('Percentage of Applications Approved for Self-Employed Applicants')
# Save the visualization to a folder in your GitHub repository
#plt.savefig('path/to/your/github/repository/self_employed_approval.png')









In [ ]:
#Req 5.2

# Filter married male applicants
married_male_applicants = loan_data[(loan_data['Gender'] == 'Male') & (loan_data['Married'] == 'Yes')]
# Calculate the percentage of rejection among married male applicants
rejection_percentage = (married_male_applicants['Application_Status'] == 'N').mean() * 100
# Create a bar chart
plt.bar(['Married Male'], [rejection_percentage])
plt.ylabel('Percentage Rejected')
plt.title('Percentage of Rejection for Married Male Applicants')
# Save the visualization to a folder in your GitHub repository
#plt.savefig('path/to/your/github/repository/married_male_rejection.png')

In [ ]:
#Req 5.3

# Assuming your dataset has a date column named 'Application_Date'
loan_data['Application_Date'] = pd.to_datetime(loan_data['Application_Date'])
# Calculate the count of applications for each month
monthly_counts = loan_data.groupby(loan_data['Application_Date'].dt.to_period('M')).size()
# Select the top three months with the largest volume
top_three_months = monthly_counts.nlargest(3)
# Create a bar chart
top_three_months.plot(kind='bar')
plt.xlabel('Month')
plt.ylabel('Number of Applications')
plt.title('Top Three Months with Largest Volume of Applications')
# Save the visualization to a folder in your GitHub repository
#plt.savefig('path/to/your/github/repository/top_three_months.png')

In [ ]:
#Req 5.4

